In [22]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


In [44]:
train_data, test_data = IMDB(split=('train', 'test'))

In [48]:
tokenizer = get_tokenizer('basic_english') 

def yield_tokens(data_iter): 
    for _, text in data_iter: 
        yield tokenizer(text) 

def get_vocab(train_datapipe): 
    vocab = build_vocab_from_iterator(yield_tokens(train_datapipe),specials=['<UNK>', '<PAD>'], max_tokens=20000) 
    vocab.set_default_index(vocab['<UNK>']) 
    return vocab 

train_data, test_data = IMDB(split=('train', 'test'))
train_vocab = get_vocab(train_data)
test_vocab = get_vocab(test_data)


In [57]:
train_vocab.get_itos()[2]

'the'

In [56]:
train_vocab['the']

2

In [42]:
vocab_size = len(vocab)

In [32]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = IMDB(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [35]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)
        
    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz, hidden_size)).to(device)
        for word in inputs:
            ht = self.rnn(word, ht)
        return ht
    

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
        self.rnn = RNNCell_Encoder(embedding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)
    
    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [43]:
model = Net()
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

NameError: name 'embedding_dim' is not defined

In [16]:

start = time.time()
train_iter = IMDB(split='train')

def tokenize(label, line):
    return line.split()



In [17]:
tokens = []
for label, line in train_iter:
    tokens += tokenize(label, line)


In [21]:
tokens

['I',
 'rented',
 'I',
 'AM',
 'CURIOUS-YELLOW',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967.',
 'I',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'U.S.',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country,',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 '"controversial"',
 'I',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself.<br',
 '/><br',
 '/>The',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'Swedish',
 'drama',
 'student',
 'named',
 'Lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life.',
 'In',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attentions',
 'to',
 'making',
 'some',
 'sort',
 'of',
 'documentary',
 'on',
 'what',
 'the',
 'average',
 'Swede',
 'thought',
 'about',
 'certai